# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [2]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn
from torch.utils.data import DataLoader

import numpy as np
import time
import math

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
-- устойчивая фраза

$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
-- устойчивая фраза - пословица

$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
-- Ваня более часто встречающееся имя

$$ P(журналы | я, часто ,читаю) > P(комиксы | я, часто ,читаю)
$$
-- смотря какую аудиторию мы рассматриваем, но мне кажется, что журналы в сумме популярнее

Ответ : 

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX' -O data.txt

--2021-12-20 20:27:59--  https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX
Resolving docs.google.com (docs.google.com)... 64.233.189.102, 64.233.189.100, 64.233.189.138, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fnhd5jnbvq8hp3kdvs87uf49nl32n0v4/1640032050000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download [following]
--2021-12-20 20:28:00--  https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fnhd5jnbvq8hp3kdvs87uf49nl32n0v4/1640032050000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download
Resolving doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to doc-0s-54-docs.googleusercontent.com (doc-0s-54-

In [4]:
file = open('data.txt', 'r')
data = file.readlines()
file.close()
len(data)

700000

In [5]:
data[:10]

['добро\n',
 'кого\n',
 'капитан\n',
 'нет\n',
 'зачем\n',
 'что происходит\n',
 'что такое\n',
 'рассказ\n',
 'никому\n',
 'ну что\n']

In [6]:
# убираем переносы и табуляцию
data = [i.replace('\n','') for i in data]
data = [i.replace('\t','') for i in data]

In [7]:
datastr = ''.join(data)
simbs = set(datastr) # сет разных символов
print(len(simbs))
print(simbs)

35
{'ч', 'ё', 'ц', 'з', 'ш', 'е', 'к', 'ж', 'щ', 'я', ' ', 'м', 'и', 'г', '-', 'ъ', 'п', 'д', 'х', 'э', 'л', 'р', 'т', 'н', 'ю', 'о', 'ф', 'ь', 'й', 'а', 'с', 'в', 'ы', 'у', 'б'}


## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [8]:
class Preprocessor:
    def __init__(self):

        self.alphabet = '_добсркгаупитнезчм яжлйвцыэь-шхющёъф][ ' # добавила букву Ф
        self.token2ind = {}
        self.ind2token = {}
        self.starttoken = '[и]' # добавляем старт и стоп токены
        self.stoptoken = '[и]'
        self.paddtoken = '_'   # пад токен
        
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    def preprocess(self, text, window_size):
        
        text = list(text.lower())
        n_padds = window_size - len(text)
        text_pads = np.hstack([text, [self.paddtoken] * n_padds])
        vec = torch.tensor([self.token2ind[i] for i in text_pads], dtype = torch.int64)
      
        return vec[:-1], vec[1:]


In [9]:
text = 'абракадабра' # проверка
print(len(text))
Preprocessor().preprocess(text, 15)

11


(tensor([8, 3, 5, 8, 6, 8, 1, 8, 3, 5, 8, 0, 0, 0]),
 tensor([3, 5, 8, 6, 8, 1, 8, 3, 5, 8, 0, 0, 0, 0]))

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [10]:
data_tok = np.char.add(['['], data)
data_tok = np.char.add(data_tok, [']'])
data_tok

array(['[добро]', '[кого]', '[капитан]', ...,
       '[хорошо хорошо спасибо всего доброго]',
       '[а если вот это клиника наша]',
       '[это в отделении позвоните или напишите]'], dtype='<U542')

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [11]:
THRESHOLD = 128

# оставляем тексты не больше порога
data_tok = data_tok[ np.char.str_len(data_tok) <= THRESHOLD]
len(data_tok)

683438

In [12]:
np.random.shuffle(data_tok)

train, test = data_tok[:int(len(data_tok)*0.85)], data_tok[int(len(data_tok)*0.85):]

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [13]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        
        x_prepro = lambda x: preproc.preprocess(x, win_size)
        self.data = list(map(x_prepro, test))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [14]:
preproc = Preprocessor()
train_dataset = TextDataset(train, preproc)
test_dataset = TextDataset(test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Когда мы используем маскирование, мы запрещаем модели на этапе применения механизма внимания в декодере "заглядывать в будущее" - использовать последующие симфолы для предсказания интересующего нас символа. Для этого мы говорим, что последующие позиции = -inf.

(thx to https://medium.com/analytics-vidhya/masking-in-transformers-self-attention-mechanism-bad3c9ec235c)

In [15]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        
        # гиперпараметры - подбор не приведен, тк был достаточно хаотичный...
        d_model = 128
        nhead = 4
        num_layers = 2
        #d_hid = 200

        self.vocab_size = vocab_size
        self.emb = nn.Embedding(num_embeddings = vocab_size,
                                embedding_dim = d_model,  padding_idx = 0)
        self.pe = PositionalEncoding(d_model = d_model, dropout = 0.3)

        self.transformer_encoder_layer =  nn.TransformerEncoderLayer(d_model = d_model, 
                                                                     nhead = nhead, dropout = 0.3)
                                                                     #dim_feedforward = d_hid )
        
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(x) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [17]:
model = LanguageModel(len('_добсркгаупитнезчм яжлйвцыэь-шхющёъ][ф '))

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [24]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset, device = "cuda:0"):
        
        self.model = model
        self.train_batch_size = 128
        self.test_batch_size = 128
        
        self.train_dataloader = DataLoader(train_dataset,
                                           batch_size = self.train_batch_size,
                                           shuffle = True)
        
        self.test_dataloader = DataLoader(train_dataset,
                                           batch_size = self.test_batch_size,
                                           shuffle = False)
        
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        
        self.device = device
        self.criterion = nn.CrossEntropyLoss(ignore_index = int(0))        
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, eps=1e-08)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number, device = "cuda:0"):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader: # по аналогии с предыдущими дз
            
            step +=1
            it +=1
            
            x, y = batch
            x, y = x.to(device), y.to(device)

            batch_size = x.size(1)
            
            self.src_mask = model.generate_square_subsequent_mask(batch_size).to(device)
                
            predicted = model(x, self.src_mask).transpose(-1,1)
            loss = self.criterion(predicted, y)
            
            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            self.optimizer.step()
            
            counted_loss += loss.item()            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number): # по аналогии с предыдущими дз
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0

        for batch in self.test_dataloader:
            it +=1
            step +=1
            x, y = batch
            x, y = x.to(torch.int64), y.to(torch.int64)
            x, y = x.to(device), y.to(device)
            batch_size = x.size(1)
            
            self.src_mask = model.generate_square_subsequent_mask(batch_size).to(device)
                
            predicted = model(x, self.src_mask).transpose(-1,1)
            loss = self.criterion(predicted, y)
            counted_loss += loss.item()
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
training = Trainer(model, train_dataset, test_dataset, device)
training.train(2)

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [ ]:
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    
    pred = ...
    ind = ... 
    
    text += ... 
    
    if ...:
        return text
    else:
        return generate_text(text)

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############